<a href="https://colab.research.google.com/github/GittleBittle/bread-and-butter/blob/main/First%20GAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **1) Importing Python Packages for GAN**


In [1]:
from keras.datasets import fashion_mnist

from keras.models import Sequential
from keras.layers import BatchNormalization
from keras.layers import Dense, Reshape, Flatten
from keras.layers.advanced_activations import LeakyReLU
from tensorflow.keras.optimizers import Adam

import numpy as np
!mkdir generated_images

mkdir: cannot create directory ‘generated_images’: File exists


## **2) Variables for Neural Networks & Data**

In [2]:
img_width = 28
img_height = 28
channels = 1
img_shape = (img_width, img_height, channels)
latent_dim = 100
adam = Adam(lr=0.0001)


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


## **3) Building Generator**





In [3]:
from keras import activations
def build_generator():
  model = Sequential()
  
  model.add(Dense(256, input_dim=latent_dim))
  model.add(LeakyReLU(alpha=0.2))
  model.add(BatchNormalization(momentum=0.8))

  model.add(Dense(512))
  model.add(LeakyReLU(alpha=0.2))
  model.add(BatchNormalization(momentum=0.8))

  model.add(Dense(1024))
  model.add(LeakyReLU(alpha=0.2))
  model.add(BatchNormalization(momentum=0.8))

  model.add(Dense(np.prod(img_shape), activation='tanh'))
  model.add(Reshape(img_shape))

  model.summary()
  return model



generator = build_generator()




Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 256)               25856     
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 256)               0         
                                                                 
 batch_normalization (BatchN  (None, 256)              1024      
 ormalization)                                                   
                                                                 
 dense_1 (Dense)             (None, 512)               131584    
                                                                 
 leaky_re_lu_1 (LeakyReLU)   (None, 512)               0         
                                                                 
 batch_normalization_1 (Batc  (None, 512)              2048      
 hNormalization)                                        

## **4) Building Discriminator**

In [4]:
def build_discriminator():
  model = Sequential()

  model.add(Flatten(input_shape=img_shape))
  model.add(Dense(512))
  model.add(LeakyReLU(alpha=0.2))
  model.add(Dense(256))
  model.add(LeakyReLU(alpha=0.2))
  model.add(Dense(1, activation='sigmoid'))
  
  model.summary()
  return model

discriminator = build_discriminator()






Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 784)               0         
                                                                 
 dense_4 (Dense)             (None, 512)               401920    
                                                                 
 leaky_re_lu_3 (LeakyReLU)   (None, 512)               0         
                                                                 
 dense_5 (Dense)             (None, 256)               131328    
                                                                 
 leaky_re_lu_4 (LeakyReLU)   (None, 256)               0         
                                                                 
 dense_6 (Dense)             (None, 1)                 257       
                                                                 
Total params: 533,505
Trainable params: 533,505
Non-tr

## **5) Connecting Neural Networks to build GAN**

In [5]:
discriminator.compile(loss='binary_crossentropy', optimizer='adam', metrics='accuracy')

GAN = Sequential()
discriminator.trainable = False
GAN.add(generator)
GAN.add(discriminator)

GAN.compile(loss='binary_crossentropy', optimizer='adam')


## **6) Outputting Images**


In [6]:
#@title
## **7) Outputting Images**
import matplotlib.pyplot as plt
import glob
import imageio
import PIL

save_name = 0.00000000

def save_imgs(epoch):
    #generate 25 images to fit on a 5 x 5 grid for our animation!
    r, c = 5, 5
    noise = np.random.normal(0, 1, (r * c, latent_dim))
    gen_imgs = generator.predict(noise)
    global save_name
    save_name += 0.00000001
    print("%.8f" % save_name)

    # Rescale images 0 - 1
    gen_imgs = 0.5 * gen_imgs + 0.5

    fig, axs = plt.subplots(r, c)
    cnt = 0
    for i in range(r):
        for j in range(c):
            axs[i,j].imshow(gen_imgs[cnt, :,:,0], cmap='gray')
            # axs[i,j].imshow(gen_imgs[cnt])
            axs[i,j].axis('off')
            cnt += 1
    fig.savefig("generated_images/%.8f.png" % save_name)
    print('saved')
    plt.close()

## **7) Training GAN**

In [11]:
def train(epochs, batch_size=64, save_interval=200):
  (X_train, _), (_, _) = fashion_mnist.load_data()
  X_train = X_train / 127.5 -1.

  valid = np.ones((batch_size, 1))
  fakes = np.zeros((batch_size, 1))

  for epoch in range(epochs):
    idx = np.random.randint(0, X_train.shape[0], batch_size)
    imgs = X_train[idx]

    noise = np.random.normal(0, 1, (batch_size, latent_dim))
    gen_imgs = generator.predict(noise)

    #Train the discriminator
    d_loss_real = discriminator.train_on_batch(imgs, valid)
    d_loss_fake = discriminator.train_on_batch(gen_imgs, fakes)
    d_loss = np.add(d_loss_real, d_loss_fake) *0.5
  
    noise = np.random.normal(0, 1, (batch_size, latent_dim))

    #Training The GANNNN
    g_loss = GAN.train_on_batch(noise, valid)
    print("********* %d [D loss: %f, acc:, %.2f] [G loss: %f]" % (epoch, d_loss[0], d_loss[1] *100, g_loss))
  if(epoch % save_interval) == 0:
      save_imgs(epoch)

 # print(X_train.shape)

train(30000, batch_size=64, save_interval=200)


Streaming output truncated to the last 5000 lines.
********* 311 [D loss: 0.031848, acc:, 98.44] [G loss: 9.247357]
********* 312 [D loss: 0.013083, acc:, 100.00] [G loss: 7.569630]
********* 313 [D loss: 0.019721, acc:, 100.00] [G loss: 6.947188]
********* 314 [D loss: 0.018079, acc:, 99.22] [G loss: 8.888227]
********* 315 [D loss: 0.006919, acc:, 100.00] [G loss: 10.865485]
********* 316 [D loss: 0.024414, acc:, 99.22] [G loss: 10.636990]
********* 317 [D loss: 0.006487, acc:, 100.00] [G loss: 9.853662]
********* 318 [D loss: 0.012895, acc:, 100.00] [G loss: 8.205372]
********* 319 [D loss: 0.013578, acc:, 100.00] [G loss: 9.132437]
********* 320 [D loss: 0.005106, acc:, 100.00] [G loss: 10.197159]
********* 321 [D loss: 0.006363, acc:, 100.00] [G loss: 10.403900]
********* 322 [D loss: 0.004798, acc:, 100.00] [G loss: 10.425439]
********* 323 [D loss: 0.052290, acc:, 99.22] [G loss: 7.524697]
********* 324 [D loss: 0.028244, acc:, 100.00] [G loss: 7.267754]
********* 325 [D loss: 0

KeyboardInterrupt: ignored

### **8) Making GIF**

In [12]:
anim_file = 'dcgan.gif'

with imageio.get_writer(anim_file, mode='I') as writer:
  filenames = glob.glob('generated_images/*.png')
  filenames = sorted(filenames)
  for filename in filenames:
    image = imageio.imread(filename)
    writer.append_data(image)
  image = imageio.imread(filename)
  writer.append_data(image)